In [ ]:
!pip install ipywidgets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 5.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

# Чтение xlsx файла в DataFrame
file_path = 'data.xlsx'  # Убедитесь, что файл загружен в рабочую директорию
df = pd.read_excel(file_path)
characteristics = df.columns.difference(['Название', 'Ссылка', 'Тип ПЛИС'])

# Преобразование значений в столбцах, начинающихся с "Наличие"
for col in characteristics:
    if col.startswith('Наличие'):
        df[col] = df[col].replace({1: 'да', 0: 'нет'})

# Заменить значения "-", "nan" и другие нежелательные значения на "неважно"
df = df.fillna("неважно")
df.replace("-", "неважно", inplace=True)

# Создаем виджеты для каждой характеристики
widgets_dict = {}
labels_dict = {}
for characteristic in characteristics:
    options = list(df[characteristic].unique())
    if "неважно" not in options:
        options.append("неважно")
    widgets_dict[characteristic] = widgets.Dropdown(
        options=options,
        layout=widgets.Layout(width='50%')
    )
    labels_dict[characteristic] = widgets.Label(
        value=characteristic,
        layout=widgets.Layout(width='50%')
    )

# Функция для обработки выбора пользователя и нахождения наиболее подходящей платы
def find_best_board(b):
    scores = {board: 0 for board in df["Название"]}

    for characteristic in characteristics:
        user_choice = widgets_dict[characteristic].value
        if user_choice != "неважно":
            for index, row in df.iterrows():
                if row[characteristic] == user_choice:
                    scores[row["Название"]] += 1

    best_board = max(scores, key=scores.get)
    print(f"Наиболее подходящая плата: {best_board}")

# Кнопка для завершения выбора и вызова функции
button = widgets.Button(description="Завершить выбор")
button.on_click(find_best_board)

# Собираем все виджеты и метки в список для отображения
widgets_list = []
for characteristic in characteristics:
    widgets_list.append(widgets.HBox([labels_dict[characteristic], widgets_dict[characteristic]]))

# Отображаем виджеты
widgets_box = widgets.VBox(widgets_list, layout=widgets.Layout(flex_flow='column', align_items='stretch', width='100%'))
display(widgets_box)

# Отображаем кнопку ниже всех виджетов
display(button)


Button(description='Завершить выбор', style=ButtonStyle())